In [73]:
import quantiacsToolbox
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

In [27]:
train_begin = '20180101'
train_end = '20201231'
df = pd.read_csv('data_'+train_begin+'_'+train_end+'.csv')
df = df.rename(columns = {'Unnamed: 0':'date'})
df['date'] = pd.to_datetime(df['date'])
df = df.drop(['date'], axis = 1)
df.head()

,CASH,F_AD,F_BO,F_BP,F_C,F_CC,F_CD,F_CL,F_CT,F_DX,...,F_RY,F_SH,F_SX,F_TR,F_EB,F_VF,F_VT,F_VW,F_GD,F_F
0,NaN,78300.0,20130.0,85156.25,17662.5,19360.0,80010.0,60370.0,38750.0,91573.0,...,16926250.0,25820.0,92690.0,39762.5,250812.5,128010.0,148800.0,112265.0,110675.0,251825.0
1,NaN,78370.0,20340.0,84681.25,17650.0,19070.0,79805.0,61970.0,39055.0,91896.0,...,16913750.0,26130.0,93810.0,40037.5,250812.5,128090.0,149030.0,112270.0,111812.5,251825.0
2,NaN,78640.0,20322.0,84887.50,17550.0,19050.0,80160.0,61890.0,39625.0,91581.0,...,17025000.0,26290.0,94290.0,40225.0,250812.5,128060.0,149110.0,112265.0,111850.0,251825.0
3,NaN,78620.0,20256.0,84981.25,17562.5,18950.0,80630.0,61590.0,39005.0,91673.0,...,17052500.0,26520.0,94440.0,40187.5,250812.5,128080.0,149090.0,112265.0,110925.0,251825.0
4,NaN,78390.0,20124.0,84987.50,17362.5,19140.0,80565.0,61900.0,39070.0,92100.0,...,16921250.0,26630.0,94470.0,39925.0,250812.5,128160.0,149300.0,112280.0,111012.5,251825.0


In [80]:
df.shape

(776, 88)

In [29]:
df = df.drop(['CASH'], axis = 1)
df.head()

,F_AD,F_BO,F_BP,F_C,F_CC,F_CD,F_CL,F_CT,F_DX,F_EC,...,F_RY,F_SH,F_SX,F_TR,F_EB,F_VF,F_VT,F_VW,F_GD,F_F
0,78300.0,20130.0,85156.25,17662.5,19360.0,80010.0,60370.0,38750.0,91573.0,151375.00,...,16926250.0,25820.0,92690.0,39762.5,250812.5,128010.0,148800.0,112265.0,110675.0,251825.0
1,78370.0,20340.0,84681.25,17650.0,19070.0,79805.0,61970.0,39055.0,91896.0,150837.50,...,16913750.0,26130.0,93810.0,40037.5,250812.5,128090.0,149030.0,112270.0,111812.5,251825.0
2,78640.0,20322.0,84887.50,17550.0,19050.0,80160.0,61890.0,39625.0,91581.0,151493.75,...,17025000.0,26290.0,94290.0,40225.0,250812.5,128060.0,149110.0,112265.0,111850.0,251825.0
3,78620.0,20256.0,84981.25,17562.5,18950.0,80630.0,61590.0,39005.0,91673.0,150975.00,...,17052500.0,26520.0,94440.0,40187.5,250812.5,128080.0,149090.0,112265.0,110925.0,251825.0
4,78390.0,20124.0,84987.50,17362.5,19140.0,80565.0,61900.0,39070.0,92100.0,150193.75,...,16921250.0,26630.0,94470.0,39925.0,250812.5,128160.0,149300.0,112280.0,111012.5,251825.0


In [3]:
markets = ['CASH','F_AD','F_BO','F_BP','F_C','F_CC','F_CD','F_CL','F_CT','F_DX','F_EC',
                           'F_ED','F_ES','F_FC','F_FV','F_GC','F_HG','F_HO','F_JY','F_KC','F_LB','F_LC',
                           'F_LN','F_MD','F_MP','F_NG','F_NQ','F_NR','F_O','F_OJ','F_PA','F_PL','F_RB',
                           'F_RU','F_S','F_SB','F_SF','F_SI','F_SM','F_TU','F_TY','F_US','F_W','F_XX',
                           'F_YM','F_AX','F_CA','F_DT','F_UB','F_UZ','F_GS','F_LX','F_SS','F_DL','F_ZQ',
                           'F_VX','F_AE','F_BG','F_BC','F_LU','F_DM','F_AH','F_CF','F_DZ','F_FB','F_FL',
                           'F_FM','F_FP','F_FY','F_GX','F_HP','F_LR','F_LQ','F_ND','F_NY','F_PQ','F_RR',
                           'F_RF','F_RP','F_RY','F_SH','F_SX','F_TR','F_EB','F_VF','F_VT','F_VW','F_GD','F_F']

# Linear Regression

In [68]:
CLOSE = df.values

In [103]:
lookback = 504
pos = np.zeros(nMarkets, dtype=np.float)
correct_dict = {el:0 for el in markets[1:]}
wrong_dict = {el:0 for el in markets[1:]}
poly = PolynomialFeatures(degree=dimension)
for i in range(len(df)-100, len(df)-1):
    for market in range(nMarkets-1):
        reg = linear_model.LinearRegression()
        correct = 0
        reg.fit(poly.fit_transform(np.arange(lookback).reshape(-1, 1)), CLOSE[i-lookback+1:i+1, market])
        trend = (reg.predict(poly.fit_transform(np.array([[lookback]]))) - CLOSE[i, market]) / CLOSE[i, market]
        if abs(trend[0]) < threshold:
            trend[0] = 0
        pos[market+1] = np.sign(trend)
        actual = 1 if df[markets[market+1]][i+1] > df[markets[market+1]][i] else -1
        if pos[market+1] > 0 and actual > 0:
            correct_dict[markets[market+1]] += 1
        elif pos[market+1] < 0 and actual < 0:
            correct_dict[markets[market+1]] += 1
        elif pos[market+1] > 0 and actual < 0:
            wrong_dict[markets[market+1]] += 1
        elif pos[market+1] < 0 and actual > 0:
            wrong_dict[markets[market+1]] += 1

In [107]:
correct_list = [0] + list(correct_dict.values())
wrong_list = [0] + list(wrong_dict.values())

In [110]:
final_list = []
for i in range(len(correct_list)):
    final_list.append(max(correct_list[i]-wrong_list[i], 0))
print(final_list)

[0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 5, 0, 0, 0, 0, 9, 0, 0, 0, 0, 4, 0, 0, 0, 5, 2, 7, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 1, 0, 0, 0, 2, 4, 0, 0, 0, 5, 4, 1, 4, 0, 5, 2, 8, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 5, 0]


In [111]:
with open('weights_list_linreg.txt', 'w') as f:
    for final in final_list:
        f.write(str(final) + '\n')